# Make the Prediction Using the Previously Trained Model

This script aims to make the prediction phase based on a previously trained LSTM model.

# 1. Importing libraries and changing working directory

Reference for the libraries:

+ [tensorflow](https://www.tensorflow.org/)
+ [numpy](https://numpy.org/)
+ [gdal](https://gdal.org/api/python.html)
+ [tqdm](https://github.com/tqdm/tqdm)
+ [matplotlib](https://matplotlib.org/)
+ [rasterio](https://rasterio.readthedocs.io/en/latest/)
+ [os](https://docs.python.org/3/library/os.html)

In [ ]:
import tensorflow as tf
import numpy as np
import gdal
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import rasterio as r
import os

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print('Tensorflow Version:', tf.__version__)

In [ ]:
# folder where all data is stored
os.chdir(os.getcwd().rsplit('/',2)[0]+'/Data')

## 2. Create the data to the prediction

Defining parameters.

In [ ]:
#####################
# Map cell
cell = '089098'
#####################
# Approach for the training samples creation. This acts only as an Identifier,
# the separation between Approaches 1, 2 and 3 must be made mannually.
sit = 'appr1'
# Cell used in training.
cell_training = '089098'
# Platform 
platform = 'Sentinel'
# State
state = 'BA'

# Year of the data.
year = 2019

# model id
identifier = '2021-07-07_14-41-34'

# Model for the stack paths.
stack_path_model = './cubes/filled/{0}.{1}.{2}.tif'
ind_fist_day = 0

# landsat: 27, sentinel: 85
if platform=='Sentinel':
    ind_last_day = 85
    stacks = ['band2', 'band3', 'band4', 'band8a', 'band11', 'band12', 'NDVI', 'EVI']
if platform=='Landsat':
    ind_last_day = 27
    stacks = ['band2', 'band3', 'band4', 'band5', 'band6', 'band7', 'NDVI', 'EVI']

nan_value = -9999

path_mask = f'./ref/PRODES_up{str(year-1)}.tif'

Creating past deforestation mask.

In [ ]:
mask = gdal.Open(path_mask).ReadAsArray()
mask = mask==0

cube = gdal.Open(stack_path_model.format(year, cell, 'band4')).ReadAsArray()

mask[cube[0,:,:]==nan_value] = False


del cube

print(mask.shape)

plt.imshow(mask)
plt.show()

Create the samples to predict upon.

In [ ]:
total_series = np.sum(mask)
total_days   = ind_last_day-ind_fist_day
total_stacks = len(stacks)

## 3. Load model

It must be changed in case some changes are made in the training script.

In [ ]:
batch_size = 256
input_dim = len(stacks)

units = 256
output_size = 2  # labels are of one dimension

# Build the RNN model
def build_model(allow_cudnn_kernel=True):
  # CuDNN is only available at the layer level, and not at the cell level.
  # This means `LSTM(units)` will use the CuDNN kernel,
  # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
  if allow_cudnn_kernel:
    # The LSTM layer with default options uses CuDNN.
    lstm_layer = tf.keras.layers.LSTM(units, input_shape=(None, input_dim))
  else:
    # Wrapping a LSTMCell in a RNN layer will not use CuDNN.
    lstm_layer = tf.keras.layers.RNN(
        tf.keras.layers.LSTMCell(units),
        input_shape=(None, input_dim))
  model = tf.keras.models.Sequential([
      lstm_layer,
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dense(output_size, activation='softmax')])
  return model

In [ ]:
model_path = './models/LSTM/training.LSTM.{}.h5'.format(identifier)

model = tf.keras.models.load_model(model_path)
model.summary()

## 4. Make the prediction

In [ ]:
# placeholder for the series to predict
series = np.zeros([total_series, total_days, total_stacks], dtype=np.float32)

# filling the placeholder
for i in range(len(stacks)):
    print('Allocating {}...'.format(stacks[i]))

    stack = gdal.Open(stack_path_model.format(year, cell, stacks[i])).ReadAsArray()
    stack = stack/10000

    count = 0
    for ii in tqdm(range(stack.shape[1])):
        for jj in range(stack.shape[2]):
            if mask[ii,jj]:
                series[count,:,i] = stack[ind_fist_day:ind_last_day,ii,jj]
                count += 1

    del stack

print('Finished extracting series!')

# making the prediction
result = np.zeros(mask.shape, dtype=np.float32)

gpu_model = build_model(allow_cudnn_kernel=True)
gpu_model.set_weights(model.get_weights())

pred = gpu_model.predict(series,
                         batch_size=256,
                         verbose=1,
                         steps=None,
                         callbacks=None,
                         max_queue_size=10,
                         workers=2,
                         use_multiprocessing=True)
aux = 0
for i in range(result.shape[0]):
    for j in range(result.shape[1]):
        if mask[i,j]:
            result[i,j] = pred[aux,1]
            aux += 1

del gpu_model

In [ ]:
pred.shape

## 5. Save results as ```.tif```

In [ ]:
pred_result_1 = result*10000

pred_result_1[np.invert(mask)]=-9999

In [ ]:
if not os.path.exists('./predictions/LSTM'):
    os.makedirs('./predictions/LSTM')

ref = gdal.Open(path_mask)
in_band = ref.GetRasterBand(1)

gtiff_driver = gdal.GetDriverByName('GTiff')
path_result = './predictions/LSTM/{3}.{0}.{1}.tr{2}.tif'.format(str(year), cell, cell_training, sit)
out_ds = gtiff_driver.Create(path_result, mask.shape[1], mask.shape[0], 1, 3, ['COMPRESS=LZW'])
out_ds.SetProjection(ref.GetProjection())
out_ds.SetGeoTransform(ref.GetGeoTransform())  

band = out_ds.GetRasterBand(1)
band.WriteArray(pred_result_1)
band.SetNoDataValue(-9999)
band.FlushCache()

out_ds = None
ref = None

print(path_result)

## 6. Show final results

In [ ]:
band = gdal.Open(path_result).ReadAsArray()
band = np.asarray(band, dtype=np.float32)
band[band==-9999] = None

plt.figure(figsize=(10,10))
plt.imshow(band, interpolation='nearest')
plt.title(path_result)
plt.show()